<a href="https://colab.research.google.com/github/PTson2207/DeploymentModel-GCP/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training someone model, then deploy ...

- Want: Save_model and then upload Google Storage
- Data: From Kaggle-Food 101
- Model(s): 10, 11, 12 classes

In [ ]:
! nvidia-smi

Wed Mar 24 09:38:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Setup some Function

In [ ]:
# thư viện
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# Hàm unzip dowload_data file

def unzip_data(file_name):
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [ ]:
# Setup dataset input
IMG_SIZE = (224, 224)

def create_data_loader(train_dir, test_dir, img_size=IMG_SIZE):
    train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode="categorical",
                                                                     image_size=img_size)
    
    test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode="categorical",
                                                                    image_size=img_size)
    return train_data, test_data

In [ ]:
data_augument = keras.Sequential([
                                  preprocessing.RandomFlip('horizontal'),
                                  preprocessing.RandomRotation(0.2),
                                  preprocessing.RandomHeight(0.2),
                                  preprocessing.RandomWidth(0.2),
                                  preprocessing.RandomZoom(0.2),
                                  #preprocessing.Rescaling(1./255) # cho Resnet50V2, không cần thiết với EfficientNetB0
], name="data_augument")

In [ ]:
# setup input_shape and base_model
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

16711680/16705208 [==============================] - 0s 0us/step


In [ ]:
def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
    # Fine tune model
    base_model.trainable = False
    # tạo layers
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
    # thêm data_augument như một layer
    x = data_augument(inputs)
    # give model inputs (after augument) nhưng không train
    x = base_model(x, training= False)
    # Pool layer
    x = layers.GlobalAveragePooling2D(name='pooling_layers')(x)
    # put a dense layer as a out_put
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layers')(x)

    # model từ input và output
    model = keras.Model(inputs, outputs)

    model.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
# Hàm import ảnh và resize ảnh
def load_and_prep_img(filename, img_shape=224, scale=False):
    # đọc ảnh
    img = tf.io.read_file(filename)
    # dịch ảnh sang tensor unit8
    img = tf.image.decode_jpeg(img)
    # thay đổi kích thước ảnh
    img = tf.image.resize(img, [img_shape, img_shape])
    # đổi giá trị ảnh về giữa đoạn 0 đến 1
    if scale:
        return img/255
    else:
        return img

Download Data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

--2021-03-24 09:38:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M   100MB/s    in 5.0s    

2021-03-24 09:38:22 (98.1 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [ ]:
# có bao nhiêu ảnh trong mỗi folder ?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
    print(f"Có {len(dirnames)} thư mục con và có {len(filenames)} ảnh trong thư mục '{dirpath}'.")

Có 2 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data'.
Có 10 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data/train'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/grilled_salmon'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/ice_cream'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/fried_rice'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/chicken_curry'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/sushi'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/chicken_wings'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/pizza'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/ramen'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/hamburger'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10

In [ ]:
# hàm tạo nhằm mục đích load lại mỗi khi chạy mới mô hình
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
                log_dir=log_dir
    )
    print(f"Lưu lại thông tin ở: {log_dir}")
    return tensorboard_callback

    Vì ta có khả năng có thể chạy lại model nhiều lần, vì thế ta có thể theo dõi chúng theo một cách nào đó.
    Hàm trên lưu lại nhật kí hiệu suất mô hình ở thư mục: [dir_name]/[experiment_name]/[current_timestamp]:
    - dir_name:  thư mục lưu lại nhật kí tổng thể.
    - experiment_name: thử nghiệm cụ thể.
    - current_timestamp: thời gian thử nghiệm chi tiết.

Model 1 (10 classes)

In [ ]:
train_data, test_data = create_data_loader(train_dir="10_food_classes_all_data/train/",
                                           test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
# xem kích cỡ dataset
print(train_data)
len(train_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>


235

In [ ]:
print(test_data)
len(test_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>


79

In [ ]:
# call model
model_1 = create_model(num_classes=len(train_data.class_names))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augument (Sequential)   (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layers (GlobalAverag (None, 1280)              0         
_________________________________________________________________
output_layers (Dense)        (None, 10)                12810     
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
history_1_percent = model_1.fit(train_data,
                    epochs=10,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)),
                    callbacks = [create_tensorboard_callback('transfer_learning', 'all_data_aug')]
                    )

Lưu lại thông tin ở: transfer_learning/all_data_aug/20210324-093829
Epoch 1/10
235/235 [==============================] - 78s 185ms/step - loss: 1.4454 - accuracy: 0.5557 - val_loss: 0.5111 - val_accuracy: 0.8405
Epoch 2/10
235/235 [==============================] - 37s 155ms/step - loss: 0.7227 - accuracy: 0.7760 - val_loss: 0.4532 - val_accuracy: 0.8569
Epoch 3/10
235/235 [==============================] - 36s 149ms/step - loss: 0.6347 - accuracy: 0.7977 - val_loss: 0.4269 - val_accuracy: 0.8520
Epoch 4/10
235/235 [==============================] - 34s 143ms/step - loss: 0.5766 - accuracy: 0.8224 - val_loss: 0.3844 - val_accuracy: 0.8750
Epoch 5/10
235/235 [==============================] - 33s 139ms/step - loss: 0.5388 - accuracy: 0.8296 - val_loss: 0.3726 - val_accuracy: 0.8750
Epoch 6/10
235/235 [==============================] - 32s 135ms/step - loss: 0.5089 - accuracy: 0.8445 - val_loss: 0.3819 - val_accuracy: 0.8701
Epoch 7/10
235/235 [==============================] - 33s 136m

In [ ]:
# số lớp(labels) có thể dự đoán được.
class_name = train_data.class_names
class_name

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [ ]:
len(class_name)

10

In [ ]:
class_name.to_json()

AttributeError: ignored

In [ ]:
# test model
# load ảnh
!wget https://img-global.cpcdn.com/recipes/249855c4b55f69ca/751x532cq70/hamburger-heo-bo-nha-lam-recipe-main-photo.jpg

--2021-03-24 09:44:47--  https://img-global.cpcdn.com/recipes/249855c4b55f69ca/751x532cq70/hamburger-heo-bo-nha-lam-recipe-main-photo.jpg
Resolving img-global.cpcdn.com (img-global.cpcdn.com)... 151.101.2.132, 151.101.66.132, 151.101.130.132, ...
Connecting to img-global.cpcdn.com (img-global.cpcdn.com)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34551 (34K) [image/jpeg]
Saving to: ‘hamburger-heo-bo-nha-lam-recipe-main-photo.jpg’

hamburger-heo-bo-nh 100%[===================>]  33.74K  --.-KB/s    in 0.001s  

2021-03-24 09:44:47 (50.4 MB/s) - ‘hamburger-heo-bo-nha-lam-recipe-main-photo.jpg’ saved [34551/34551]



In [ ]:
# Xử lí lại ảnh
hamburger_img = load_and_prep_img("hamburger-heo-bo-nha-lam-recipe-main-photo.jpg")

In [ ]:
hamburger_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[162.55135 , 166.55135 , 167.55135 ],
        [165.54031 , 169.54031 , 170.54031 ],
        [163.76688 , 167.76688 , 168.76688 ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       [[167.9375  , 171.9375  , 174.9375  ],
        [166.96693 , 170.96693 , 173.96693 ],
        [166.9926  , 170.9926  , 173.9926  ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       [[167.07715 , 170.63965 , 174.51465 ],
        [168.10645 , 171.66895 , 175.54395 ],
        [168.86021 , 172.42271 , 176.29771 ],
        ...,
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ],
        [255.      , 255.      , 255.      ]],

       ...,

       [[101.23535 ,  96.23535 , 102.23535 ],
        [100.

In [ ]:
# mở rổng ảnh
hamburger_expanded = tf.expand_dims(hamburger_img, axis=0) # mở rộng kích thước ảnh từ (224, 224, 3) --> (1, 224, 224, 3)
predict = model_1.predict(hamburger_expanded)
predict

array([[2.1485635e-06, 1.5014882e-07, 1.5500862e-05, 5.9190256e-06,
        9.9687445e-01, 3.0901586e-03, 5.3037235e-07, 2.3895615e-07,
        4.5432736e-07, 1.0506758e-05]], dtype=float32)

In [ ]:
# kiểm tra nó thuộc lớp nào
class_name[tf.argmax(predict[0])]

'hamburger'

Save and  Upload model_1 to Google Storage

In [ ]:
model_1.save("efficientnet_model_1_10_classes.h5")

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

In [ ]:
#!curl https://sdk.cloud.google.com | bash 1> /dev/null
#!gcloud init

Model_2 (11 classes)

In [ ]:
# Download data
! wget https://storage.googleapis.com/ml_deploymet_lec_bucket_0/11_food_classes_all_data.zip

--2021-03-24 09:44:49--  https://storage.googleapis.com/ml_deploymet_lec_bucket_0/11_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 570914387 (544M) [application/zip]
Saving to: ‘11_food_classes_all_data.zip’

11_food_classes_all 100%[===================>] 544.47M  87.3MB/s    in 6.5s    

2021-03-24 09:44:56 (83.1 MB/s) - ‘11_food_classes_all_data.zip’ saved [570914387/570914387]



In [ ]:
unzip_data("11_food_classes_all_data.zip")

In [ ]:
train_data, test_data = create_data_loader(train_dir="/content/11_food_classes_all_data/train/",
                                           test_dir="/content/11_food_classes_all_data/test/")

Found 8250 files belonging to 11 classes.
Found 2750 files belonging to 11 classes.


In [ ]:
train_data.class_names

['chicken_curry',
 'chicken_wings',
 'donuts',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [ ]:
len(train_data.class_names)

11

In [ ]:
model_2 = create_model(num_classes=len(train_data.class_names))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augument (Sequential)   (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layers (GlobalAverag (None, 1280)              0         
_________________________________________________________________
output_layers (Dense)        (None, 11)                14091     
Total params: 4,063,662
Trainable params: 14,091
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
history_2 = model_2.fit(train_data,
                        epochs=10,
                        validation_data=test_data,
                        validation_steps=int(0.25*len(test_data)))

Epoch 1/10
258/258 [==============================] - 39s 132ms/step - loss: 1.4784 - accuracy: 0.5609 - val_loss: 0.5503 - val_accuracy: 0.8333
Epoch 2/10
258/258 [==============================] - 33s 128ms/step - loss: 0.7354 - accuracy: 0.7721 - val_loss: 0.4795 - val_accuracy: 0.8438
Epoch 3/10
258/258 [==============================] - 33s 126ms/step - loss: 0.6284 - accuracy: 0.8112 - val_loss: 0.4044 - val_accuracy: 0.8631
Epoch 4/10
258/258 [==============================] - 33s 127ms/step - loss: 0.5682 - accuracy: 0.8195 - val_loss: 0.4107 - val_accuracy: 0.8616
Epoch 5/10
127/258 [=============>................] - ETA: 15s - loss: 0.5573 - accuracy: 0.8289

KeyboardInterrupt: ignored

In [ ]:
# predict
! wget https://cdn.tgdd.vn/Files/2018/07/02/1098913/ca-nha-cung-lam-banh-donut-sieu-ngon--sieu-de-thuong-6.JPG

In [ ]:
donut_img = load_and_prep_img("/content/ca-nha-cung-lam-banh-donut-sieu-ngon--sieu-de-thuong-6.JPG")

In [ ]:
donut_expand = tf.expand_dims(donut_img, axis=0)
predict = model_2.predict(donut_expand)

In [ ]:
class_name = train_data.class_names

In [ ]:
class_name[tf.argmax(predict[0])]